# CAE模型特征融合 - Xpeng和BYD Cradle


- Replace Xpeng Cross Member to BYD Cross Member 



In [1]:
import os
import sys
import json
import numpy as np
import torch
import matplotlib
matplotlib.use('Agg')  # 使用非交互式后端
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 添加项目路径
project_root = '/home/zong/Project/gensdf'
sys.path.insert(0, project_root)
sys.path.insert(0, os.path.join(project_root, 'data', 'preprocessing'))
# 导入自定义模块
from model import GenSDF
from utils.pointcloud_normalize import normalize_pointcloud, save_transform_params, apply_transform
from utils.sdf_fusion import SDFFusion
from data.preprocessing.cae_to_csv import parse_cae_file


/home/zong/miniconda3/envs/sdf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 步骤 1: 定义文件路径和输出目录


In [2]:
# 定义数据路径
data_dir = "/home/zong/Project/gensdf/data/CAE/Model_Split"

# Xpeng模型文件
xpeng_clean = os.path.join(data_dir, "0584_Xpeng_F30_Rear_Cradle_mesh_DV_Clean.nas")
xpeng_frt_rr = os.path.join(data_dir, "0584_Xpeng_F30_Rear_Cradle_mesh_DV_Frt_Rr.nas")
xpeng_lateral = os.path.join(data_dir, "0584_Xpeng_F30_Rear_Cradle_mesh_DV_Lateral.nas")

# BYD模型文件
byd_clean = os.path.join(data_dir, "0761_BYD_UNE_AC01_Rear_Cradle_Clean.nas")
byd_frt_rr = os.path.join(data_dir, "0761_BYD_UNE_AC01_Rear_Cradle_Frt_Rr.nas")
byd_side = os.path.join(data_dir, "0761_BYD_UNE_AC01_Rear_Cradle_Side.nas")

# 输出目录
output_dir = "/home/zong/Project/gensdf/output_directory/fusion_two_cradle"
os.makedirs(output_dir, exist_ok=True)



## 步骤 2: 加载所有CAE文件


In [3]:
# 在 notebook 中重新加载模块
import importlib
import sys

# 如果模块已经在 sys.modules 中，需要重新加载
if 'data.preprocessing.cae_to_csv' in sys.modules:
    importlib.reload(sys.modules['data.preprocessing.cae_to_csv'])
elif 'cae_to_csv' in sys.modules:
    importlib.reload(sys.modules['cae_to_csv'])

# 重新导入
from data.preprocessing.cae_to_csv import parse_cae_file


In [3]:

pc_xpeng_clean = parse_cae_file(xpeng_clean)
print(f"✓ Xpeng完整模型: {pc_xpeng_clean.shape[0]:,} 个点")

pc_xpeng_frt_rr = parse_cae_file(xpeng_frt_rr)
print(f"✓ Xpeng前后特征: {pc_xpeng_frt_rr.shape[0]:,} 个点")

pc_xpeng_lateral = parse_cae_file(xpeng_lateral)
print(f"✓ Xpeng侧面特征: {pc_xpeng_lateral.shape[0]:,} 个点")

pc_byd_clean = parse_cae_file(byd_clean)
print(f"✓ BYD完整模型: {pc_byd_clean.shape[0]:,} 个点")

pc_byd_frt_rr = parse_cae_file(byd_frt_rr)
print(f"✓ BYD前后特征: {pc_byd_frt_rr.shape[0]:,} 个点")

pc_byd_side = parse_cae_file(byd_side)
print(f"✓ BYD侧面特征: {pc_byd_side.shape[0]:,} 个点")


✓ Xpeng完整模型: 61,621 个点
✓ Xpeng前后特征: 41,036 个点
✓ Xpeng侧面特征: 20,028 个点
✓ BYD完整模型: 94,902 个点
✓ BYD前后特征: 58,375 个点
✓ BYD侧面特征: 36,563 个点


## 步骤 3: 可视化原始点云（归一化前）


In [4]:
# 可视化原始点云
print("Generating original point cloud visualization...")

fig = plt.figure(figsize=(18, 10))

# 子图1: Xpeng完整模型
ax1 = fig.add_subplot(231, projection='3d')
ax1.scatter(pc_xpeng_clean[::50, 0], pc_xpeng_clean[::50, 1], pc_xpeng_clean[::50, 2], 
            c='blue', alpha=0.3, s=1)
ax1.set_title('Xpeng Complete Model')

# 子图2: Xpeng前后特征
ax2 = fig.add_subplot(232, projection='3d')
ax2.scatter(pc_xpeng_frt_rr[::20, 0], pc_xpeng_frt_rr[::20, 1], pc_xpeng_frt_rr[::20, 2], 
            c='green', alpha=0.3, s=1)
ax2.set_title('Xpeng Front/Rear Features (Keep)')


# 子图3: Xpeng侧面特征
ax3 = fig.add_subplot(233, projection='3d')
ax3.scatter(pc_xpeng_lateral[::10, 0], pc_xpeng_lateral[::10, 1], pc_xpeng_lateral[::10, 2], 
            c='red', alpha=0.3, s=1)
ax3.set_title('Xpeng Lateral Features (Remove)')


# 子图4: BYD完整模型
ax4 = fig.add_subplot(234, projection='3d')
ax4.scatter(pc_byd_clean[::50, 0], pc_byd_clean[::50, 1], pc_byd_clean[::50, 2], 
            c='orange', alpha=0.3, s=1)
ax4.set_title('BYD Complete Model')


# 子图5: BYD前后特征
ax5 = fig.add_subplot(235, projection='3d')
ax5.scatter(pc_byd_frt_rr[::20, 0], pc_byd_frt_rr[::20, 1], pc_byd_frt_rr[::20, 2], 
            c='purple', alpha=0.3, s=1)
ax5.set_title('BYD Front/Rear Features')

# 子图6: BYD侧面特征
ax6 = fig.add_subplot(236, projection='3d')
ax6.scatter(pc_byd_side[::10, 0], pc_byd_side[::10, 1], pc_byd_side[::10, 2], 
            c='cyan', alpha=0.3, s=1)
ax6.set_title('BYD Lateral Features (For Fusion)')


plt.tight_layout()
output_path = os.path.join(output_dir, '01_original_pointclouds.png')
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"✓ Saved original point cloud visualization: {output_path}")
plt.close()

# 打印统计信息
print("\nOriginal Point Cloud Statistics:")
print(f"  Xpeng Range: X[{pc_xpeng_clean[:, 0].min():.2f}, {pc_xpeng_clean[:, 0].max():.2f}], "
      f"Y[{pc_xpeng_clean[:, 1].min():.2f}, {pc_xpeng_clean[:, 1].max():.2f}], "
      f"Z[{pc_xpeng_clean[:, 2].min():.2f}, {pc_xpeng_clean[:, 2].max():.2f}]")
print(f"  BYD Range:   X[{pc_byd_clean[:, 0].min():.2f}, {pc_byd_clean[:, 0].max():.2f}], "
      f"Y[{pc_byd_clean[:, 1].min():.2f}, {pc_byd_clean[:, 1].max():.2f}], "
      f"Z[{pc_byd_clean[:, 2].min():.2f}, {pc_byd_clean[:, 2].max():.2f}]")

Generating original point cloud visualization...
✓ Saved original point cloud visualization: /home/zong/Project/gensdf/output_directory/fusion_two_cradle/01_original_pointclouds.png

Original Point Cloud Statistics:
  Xpeng Range: X[4104.00, 4805.00], Y[-558.00, 558.00], Z[772.18, 1080.99]
  BYD Range:   X[2323.90, 3044.15], Y[-613.98, 613.98], Z[-116.75, 186.42]


In [5]:
# 基于Xpeng完整模型计算归一化参数
pc_xpeng_clean_norm, xpeng_transform_params = normalize_pointcloud(pc_xpeng_clean, return_params=True)
print(f" Xpeng Normalizing:")
print(f"  CG: {xpeng_transform_params['centroid']}")
print(f"  scale: {xpeng_transform_params['scale']:.4f}")

# 应用相同的归一化参数到Xpeng的其他部分
pc_xpeng_frt_rr_norm = apply_transform(pc_xpeng_frt_rr, xpeng_transform_params)
pc_xpeng_lateral_norm = apply_transform(pc_xpeng_lateral, xpeng_transform_params)


# 基于BYD完整模型计算归一化参数
pc_byd_clean_norm, byd_transform_params = normalize_pointcloud(pc_byd_clean, return_params=True)
print(f" BYD Normalizing")
print(f"  CG: {byd_transform_params['centroid']}")
print(f"  scale: {byd_transform_params['scale']:.4f}")

# 应用相同的归一化参数到BYD的其他部分
pc_byd_frt_rr_norm = apply_transform(pc_byd_frt_rr, byd_transform_params)
pc_byd_side_norm = apply_transform(pc_byd_side, byd_transform_params)


save_transform_params(xpeng_transform_params, os.path.join(output_dir, 'xpeng_transform.json'))
save_transform_params(byd_transform_params, os.path.join(output_dir, 'byd_transform.json'))

print("Normalizing Done!")


 Xpeng Normalizing:
  CG: [4461.721887976156, 0.3731786125919673, 928.773479459908]
  scale: 676.7968
 BYD Normalizing
  CG: [2672.2810097051442, 0.9794969221278057, 46.072431515077525]
  scale: 727.7600
Transformation parameters saved to: /home/zong/Project/gensdf/output_directory/fusion_two_cradle/xpeng_transform.json
Transformation parameters saved to: /home/zong/Project/gensdf/output_directory/fusion_two_cradle/byd_transform.json
Normalizing Done!


## 步骤 5: 可视化归一化后的点云


In [6]:


fig = plt.figure(figsize=(18, 10))

# 子图1: Xpeng完整模型（归一化）
ax1 = fig.add_subplot(231, projection='3d')
ax1.scatter(pc_xpeng_clean_norm[::50, 0], pc_xpeng_clean_norm[::50, 1], pc_xpeng_clean_norm[::50, 2], 
            c='blue', alpha=0.3, s=1)
ax1.set_title('Xpeng Normalized')


# 子图2: Xpeng前后特征（归一化）
ax2 = fig.add_subplot(232, projection='3d')
ax2.scatter(pc_xpeng_frt_rr_norm[::20, 0], pc_xpeng_frt_rr_norm[::20, 1], pc_xpeng_frt_rr_norm[::20, 2], 
            c='green', alpha=0.3, s=1)



# 子图3: Xpeng侧面特征（归一化）
ax3 = fig.add_subplot(233, projection='3d')
ax3.scatter(pc_xpeng_lateral_norm[::10, 0], pc_xpeng_lateral_norm[::10, 1], pc_xpeng_lateral_norm[::10, 2], 
            c='red', alpha=0.3, s=1)



# 子图4: BYD完整模型（归一化）
ax4 = fig.add_subplot(234, projection='3d')
ax4.scatter(pc_byd_clean_norm[::50, 0], pc_byd_clean_norm[::50, 1], pc_byd_clean_norm[::50, 2], 
            c='orange', alpha=0.3, s=1)
ax4.set_title('BYD Normalized')


# 子图5: BYD前后特征（归一化）
ax5 = fig.add_subplot(235, projection='3d')
ax5.scatter(pc_byd_frt_rr_norm[::20, 0], pc_byd_frt_rr_norm[::20, 1], pc_byd_frt_rr_norm[::20, 2], 
            c='purple', alpha=0.3, s=1)



# 子图6: BYD侧面特征（归一化）
ax6 = fig.add_subplot(236, projection='3d')
ax6.scatter(pc_byd_side_norm[::10, 0], pc_byd_side_norm[::10, 1], pc_byd_side_norm[::10, 2], 
            c='cyan', alpha=0.3, s=1)



plt.tight_layout()
output_path = os.path.join(output_dir, '02_normalized_pointclouds.png')
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"✓ 保存归一化点云可视化: {output_path}")
plt.close()

# 保存归一化后的点云到文件
np.savetxt(os.path.join(output_dir, 'xpeng_frt_rr_normalized.xyz'), pc_xpeng_frt_rr_norm, fmt='%.6f')
np.savetxt(os.path.join(output_dir, 'byd_side_normalized.xyz'), pc_byd_side_norm, fmt='%.6f')
print(f"✓ 保存归一化点云文件到: {output_dir}")


✓ 保存归一化点云可视化: /home/zong/Project/gensdf/output_directory/fusion_two_cradle/02_normalized_pointclouds.png
✓ 保存归一化点云文件到: /home/zong/Project/gensdf/output_directory/fusion_two_cradle


## 步骤 6: 加载GenSDF模型

加载训练好的GenSDF模型，用于编码点云特征和解码融合后的SDF。


In [7]:
# 加载GenSDF模型
print("正在加载GenSDF模型...")
print("-" * 70)

model_dir = "/home/zong/Project/gensdf/config/gensdf/semi"
specs_path = os.path.join(model_dir, "specs.json")

# 加载模型配置
with open(specs_path, 'r') as f:
    specs = json.load(f)
print(f"✓ 加载模型配置: {specs_path}")

# 初始化模型
model = GenSDF(specs, None).cuda()
print(f"✓ 初始化GenSDF模型")

# 加载checkpoint
checkpoint_path = os.path.join(model_dir, "last.ckpt")
checkpoint = torch.load(checkpoint_path, map_location='cuda')
model.load_state_dict(checkpoint['state_dict'])
model.eval()
print(f"✓ 加载模型权重: {checkpoint_path}")

print("-" * 70)
print("模型加载完成!")
print(f"模型已设置为评估模式")


正在加载GenSDF模型...
----------------------------------------------------------------------
✓ 加载模型配置: /home/zong/Project/gensdf/config/gensdf/semi/specs.json
✓ 初始化GenSDF模型
✓ 加载模型权重: /home/zong/Project/gensdf/config/gensdf/semi/last.ckpt
----------------------------------------------------------------------
模型加载完成!
模型已设置为评估模式


## 步骤 7: 编码点云特征

使用GenSDF的编码器将归一化后的点云编码为隐式SDF表示。


In [8]:
# 创建SDF融合实例
print("正在编码点云特征...")
print("-" * 70)

fusion = SDFFusion(model)

# 转换为PyTorch张量
# 我们要融合的两个部分:
# 1. Xpeng的前后特征 (保留部分)
# 2. BYD的侧面特征 (要添加的部分)

pc_xpeng_frt_rr_tensor = torch.from_numpy(pc_xpeng_frt_rr_norm).float()
pc_byd_side_tensor = torch.from_numpy(pc_byd_side_norm).float()

print(f"✓ Xpeng前后特征张量: {pc_xpeng_frt_rr_tensor.shape}")
print(f"✓ BYD侧面特征张量: {pc_byd_side_tensor.shape}")

# 编码两个点云
print("\n编码Xpeng前后特征...")
sdf_xpeng_frt_rr = fusion.encode_pointcloud(pc_xpeng_frt_rr_tensor)
print("✓ Xpeng前后特征编码完成")

print("编码BYD侧面特征...")
sdf_byd_side = fusion.encode_pointcloud(pc_byd_side_tensor)
print("✓ BYD侧面特征编码完成")



正在编码点云特征...
----------------------------------------------------------------------
✓ Xpeng前后特征张量: torch.Size([41036, 3])
✓ BYD侧面特征张量: torch.Size([36563, 3])

编码Xpeng前后特征...
✓ Xpeng前后特征编码完成
编码BYD侧面特征...
✓ BYD侧面特征编码完成


## 步骤 8: 可视化编码后的SDF切片

在融合之前，我们先可视化一下单独编码后的SDF值，在2D切片上展示。


In [12]:
# 可视化SDF切片
print("生成SDF切片可视化...")

# 在z=0平面创建一个2D网格
resolution = 200
x = np.linspace(-1, 1, resolution)
y = np.linspace(-1, 1, resolution)
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)

# 创建查询点
query_points = np.stack([X.flatten(), Y.flatten(), Z.flatten()], axis=-1)

# 查询SDF值
print("  查询Xpeng前后特征SDF值...")
sdf_xpeng_values = sdf_xpeng_frt_rr(query_points).reshape(X.shape)

print("  查询BYD侧面特征SDF值...")
sdf_byd_values = sdf_byd_side(query_points).reshape(X.shape)

# 创建可视化
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Xpeng前后特征
im1 = axes[0].contourf(X, Y, sdf_xpeng_values, levels=20, cmap='RdBu')
axes[0].contour(X, Y, sdf_xpeng_values, levels=[0], colors='black', linewidths=2)
axes[0].set_title('Xpeng frt Rear Feature, levels=[0] Slice')

plt.colorbar(im1, ax=axes[0], label='SDF')

# BYD侧面特征
im2 = axes[1].contourf(X, Y, sdf_byd_values, levels=20, cmap='RdBu')
axes[1].contour(X, Y, sdf_byd_values, levels=[0], colors='black', linewidths=2)
axes[1].set_title('BYD Lateral SDF Feature, levels=[0] Slice')

plt.colorbar(im2, ax=axes[1], label='SDF')

plt.tight_layout()
output_path = os.path.join(output_dir, '03_encoded_sdf_slices.png')
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"✓ 保存SDF切片可视化: {output_path}")
plt.close()


生成SDF切片可视化...
  查询Xpeng前后特征SDF值...
  查询BYD侧面特征SDF值...
✓ 保存SDF切片可视化: /home/zong/Project/gensdf/output_directory/fusion_two_cradle/03_encoded_sdf_slices.png


## 步骤 9: 融合SDF特征

使用两种不同的融合方法：
1. **Union (并集)**: 取两个SDF的最小值，产生硬边缘
2. **Blend (混合)**: 使用指数加权进行平滑混合


In [14]:
# 融合两个SDF
from utils.sdf_fusion import sdf_union, sdf_blend


# 方法1: Union (并集) - 取最小值
print("创建Union融合...")
def fused_sdf_union(x):
    return sdf_union(x, sdf_xpeng_frt_rr, sdf_byd_side)

# 方法2: Blend (混合) - 平滑过渡
print("创建Blend融合 (alpha=10)...")
def fused_sdf_blend(x):
    return sdf_blend(x, sdf_xpeng_frt_rr, sdf_byd_side, alpha=10)




创建Union融合...
创建Blend融合 (alpha=10)...


## 步骤 10: 可视化融合后的SDF


In [15]:
# 可视化融合后的SDF
print("生成融合SDF可视化...")

# 查询融合后的SDF值（使用之前创建的查询点）
print("  查询Union融合SDF值...")
sdf_union_values = fused_sdf_union(query_points).reshape(X.shape)

print("  查询Blend融合SDF值...")
sdf_blend_values = fused_sdf_blend(query_points).reshape(X.shape)

# 创建对比可视化
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# 原始Xpeng前后特征
im1 = axes[0, 0].contourf(X, Y, sdf_xpeng_values, levels=20, cmap='RdBu')
axes[0, 0].contour(X, Y, sdf_xpeng_values, levels=[0], colors='black', linewidths=2)
axes[0, 0].set_title('Original: Xpeng frt Rear Feature')

plt.colorbar(im1, ax=axes[0, 0])

# 原始BYD侧面特征
im2 = axes[0, 1].contourf(X, Y, sdf_byd_values, levels=20, cmap='RdBu')
axes[0, 1].contour(X, Y, sdf_byd_values, levels=[0], colors='black', linewidths=2)
axes[0, 1].set_title('Original: BYD Lateral Feature')

plt.colorbar(im2, ax=axes[0, 1])

# Union融合
im3 = axes[1, 0].contourf(X, Y, sdf_union_values, levels=20, cmap='RdBu')
axes[1, 0].contour(X, Y, sdf_union_values, levels=[0], colors='black', linewidths=2)
axes[1, 0].set_title('Fusion Result: Union')

plt.colorbar(im3, ax=axes[1, 0])

# Blend融合
im4 = axes[1, 1].contourf(X, Y, sdf_blend_values, levels=20, cmap='RdBu')
axes[1, 1].contour(X, Y, sdf_blend_values, levels=[0], colors='black', linewidths=2)
axes[1, 1].set_title('Fusion Result: Blender( α=10)')

plt.colorbar(im4, ax=axes[1, 1])

plt.tight_layout()
output_path = os.path.join(output_dir, '04_fused_sdf_comparison.png')
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"✓ 保存融合SDF对比可视化: {output_path}")
plt.close()


生成融合SDF可视化...
  查询Union融合SDF值...
  查询Blend融合SDF值...
✓ 保存融合SDF对比可视化: /home/zong/Project/gensdf/output_directory/fusion_two_cradle/04_fused_sdf_comparison.png


## 步骤 11: 重建3D网格 (Union方法)

使用Marching Cubes算法从融合后的SDF重建3D网格模型。


In [ ]:
# 重建Union融合的3D网格
print("=" * 70)
print("重建Union融合的3D网格...")
print("=" * 70)

output_mesh_union = os.path.join(output_dir, "fused_union_r_1024.ply")
fusion.reconstruct_from_sdf(
    fused_sdf_union, 
    output_mesh_union, 
    resolution=768,  # 可以调整分辨率，更高的值会产生更细致的网格但需要更多时间
    batch_size=64000
)

print(f"✓ Union融合网格已保存: {output_mesh_union}")
print("=" * 70)


## 步骤 12: 重建3D网格 (Blend方法)


In [ ]:
# 重建Blend融合的3D网格
print("=" * 70)
print("重建Blend融合的3D网格...")
print("=" * 70)

output_mesh_blend = os.path.join(output_dir, "fused_blend_r_512.ply")
fusion.reconstruct_from_sdf(
    fused_sdf_blend, 
    output_mesh_blend, 
    resolution=512,
    batch_size=64000
)

print(f"✓ Blend融合网格已保存: {output_mesh_blend}")
print("=" * 70)


## 步骤 13: 可视化重建的3D网格


In [ ]:
# 可视化重建的网格
import trimesh

print("生成重建网格可视化...")

# 加载两个重建的网格
mesh_union = trimesh.load(output_mesh_union)
mesh_blend = trimesh.load(output_mesh_blend)

print(f"  Union网格: {len(mesh_union.vertices)} 个顶点, {len(mesh_union.faces)} 个面")
print(f"  Blend网格: {len(mesh_blend.vertices)} 个顶点, {len(mesh_blend.faces)} 个面")

# 创建可视化
fig = plt.figure(figsize=(18, 8))

# 子图1: Union融合结果
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_trisurf(mesh_union.vertices[:, 0], 
                 mesh_union.vertices[:, 1], 
                 mesh_union.vertices[:, 2],
                 triangles=mesh_union.faces,
                 cmap='viridis',
                 alpha=0.8,
                 edgecolor='none')
ax1.set_title('Union融合重建结果', fontsize=14, fontweight='bold')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_xlim([-1, 1])
ax1.set_ylim([-1, 1])
ax1.set_zlim([-1, 1])

# 子图2: Blend融合结果
ax2 = fig.add_subplot(122, projection='3d')
ax2.plot_trisurf(mesh_blend.vertices[:, 0], 
                 mesh_blend.vertices[:, 1], 
                 mesh_blend.vertices[:, 2],
                 triangles=mesh_blend.faces,
                 cmap='plasma',
                 alpha=0.8,
                 edgecolor='none')
ax2.set_title('Blend融合重建结果', fontsize=14, fontweight='bold')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')
ax2.set_xlim([-1, 1])
ax2.set_ylim([-1, 1])
ax2.set_zlim([-1, 1])

plt.tight_layout()
output_path = os.path.join(output_dir, '05_reconstructed_meshes.png')
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"✓ 保存重建网格可视化: {output_path}")
plt.close()


In [ ]:
# 生成总结报告
print("\n" + "=" * 70)
print("融合流程完成!")
print("=" * 70)

print("\n输入数据:")
print(f"  • Xpeng前后特征: {pc_xpeng_frt_rr.shape[0]:,} 个点")
print(f"  • BYD侧面特征: {pc_byd_side.shape[0]:,} 个点")

print("\n归一化参数:")
print(f"  • Xpeng缩放因子: {xpeng_transform_params['scale']:.4f}")
print(f"  • BYD缩放因子: {byd_transform_params['scale']:.4f}")

print("\n输出文件:")
print(f"  • Union融合网格: {os.path.basename(output_mesh_union)}")
print(f"    - 顶点数: {len(mesh_union.vertices):,}")
print(f"    - 面数: {len(mesh_union.faces):,}")
print(f"  • Blend融合网格: {os.path.basename(output_mesh_blend)}")
print(f"    - 顶点数: {len(mesh_blend.vertices):,}")
print(f"    - 面数: {len(mesh_blend.faces):,}")

print("\n生成的可视化图像:")
viz_files = [f for f in os.listdir(output_dir) if f.endswith('.png')]
for i, f in enumerate(sorted(viz_files), 1):
    print(f"  {i}. {f}")

print(f"\n所有输出保存在: {output_dir}")
print("\n可以使用以下命令查看3D模型:")
print(f"  meshlab {output_mesh_union}")
print(f"  meshlab {output_mesh_blend}")
print("=" * 70)


## 可选: 1D剖面对比

查看沿着某个轴的SDF值变化，更清晰地展示融合效果。


In [13]:
# 创建1D剖面对比
print("生成1D剖面对比...")

# 沿x轴采样
x_line = np.linspace(-1, 1, 300)
points_line = np.stack([x_line, np.zeros_like(x_line), np.zeros_like(x_line)], axis=-1)

# 查询各个SDF值
sdf_xpeng_line = sdf_xpeng_frt_rr(points_line)
sdf_byd_line = sdf_byd_side(points_line)
sdf_union_line = fused_sdf_union(points_line)
sdf_blend_line = fused_sdf_blend(points_line)

# 创建图表
plt.figure(figsize=(14, 6))

plt.plot(x_line, sdf_xpeng_line, 'b-', label='Xpeng前后特征', linewidth=2)
plt.plot(x_line, sdf_byd_line, 'r-', label='BYD侧面特征', linewidth=2)
plt.plot(x_line, sdf_union_line, 'g-', label='Union融合', linewidth=2, linestyle='--')
plt.plot(x_line, sdf_blend_line, 'm-', label='Blend融合 (α=10)', linewidth=2, linestyle='--')

plt.axhline(y=0, color='k', linestyle=':', linewidth=1, label='零水平面 (表面)')
plt.xlabel('X轴位置', fontsize=12)
plt.ylabel('SDF值', fontsize=12)
plt.title('SDF融合方法 - 1D剖面对比 (y=0, z=0)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

output_path = os.path.join(output_dir, '06_sdf_profile_1d.png')
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"✓ 保存1D剖面对比: {output_path}")
plt.close()


生成1D剖面对比...
✓ 保存1D剖面对比: /home/zong/Project/gensdf/output_directory/fusion_two_cradle/06_sdf_profile_1d.png


/tmp/ipykernel_40804/1300676218.py:19: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "g-" (-> linestyle='-'). The keyword argument will take precedence.
  plt.plot(x_line, sdf_union_line, 'g-', label='Union融合', linewidth=2, linestyle='--')
/tmp/ipykernel_40804/1300676218.py:20: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "m-" (-> linestyle='-'). The keyword argument will take precedence.
  plt.plot(x_line, sdf_blend_line, 'm-', label='Blend融合 (α=10)', linewidth=2, linestyle='--')
/tmp/ipykernel_40804/1300676218.py:30: UserWarning: Glyph 20540 (\N{CJK UNIFIED IDEOGRAPH-503C}) missing from current font.
  plt.savefig(output_path, dpi=150, bbox_inches='tight')
/tmp/ipykernel_40804/1300676218.py:30: UserWarning: Glyph 34701 (\N{CJK UNIFIED IDEOGRAPH-878D}) missing from current font.
  plt.savefig(output_path, dpi=150, bbox_inches='tight')
/tmp/ipykernel_40804/1300676218.py:30: Us

# 直接重建

In [18]:
# 创建SDF融合实例
print("正在编码点云特征...")
print("-" * 70)

fusion = SDFFusion(model)

# 转换为PyTorch张量
# 我们要融合的两个部分:
# 1. Xpeng的前后特征 (保留部分)
# 2. BYD的侧面特征 (要添加的部分)

pc_xpeng_tensor = torch.from_numpy(pc_xpeng_clean).float()
pc_byd_tensor = torch.from_numpy(pc_byd_clean).float()

print(f"✓ Xpeng特征张量: {pc_xpeng_clean.shape}")
print(f"✓ BYD特征张量: {pc_byd_clean.shape}")


sdf_xpeng = fusion.encode_pointcloud(pc_xpeng_tensor)
print("✓ Xpeng特征编码完成")


sdf_byd = fusion.encode_pointcloud(pc_byd_tensor)
print("✓ BYD特征编码完成")


print("特征编码完成!")


正在编码点云特征...
----------------------------------------------------------------------
✓ Xpeng特征张量: (61621, 3)
✓ BYD特征张量: (94902, 3)
✓ Xpeng特征编码完成
✓ BYD特征编码完成
特征编码完成!


In [19]:
# ============================================================================
# 验证编码-解码过程：重建单个模型
# ============================================================================
print("\n" + "=" * 70)
print("验证编码-解码过程：从单个点云重建网格")
print("=" * 70)

# 重建Xpeng前后特征模型
print("\n重建Xpeng前后特征模型...")
output_mesh_xpeng = os.path.join(output_dir, "xpeng_reconstructed.ply")
fusion.reconstruct_from_sdf(
    sdf_xpeng, 
    output_mesh_xpeng, 
    resolution=512,  # 可以调整分辨率
    batch_size=64000
)
print(f"✓ Xpeng重建网格已保存: {output_mesh_xpeng}")

# 重建BYD侧面特征模型
print("\n重建BYD侧面特征模型...")
output_mesh_byd = os.path.join(output_dir, "byd_reconstructed.ply")
fusion.reconstruct_from_sdf(
    sdf_byd, 
    output_mesh_byd, 
    resolution=512,
    batch_size=64000
)
print(f"✓ BYD重建网格已保存: {output_mesh_byd}")

print("=" * 70)


验证编码-解码过程：从单个点云重建网格

重建Xpeng前后特征模型...
Reconstructing mesh with resolution=512...


/home/zong/Project/gensdf/utils/sdf_fusion.py:193: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  samples[:, 1] = (overall_index // N) % N
/home/zong/Project/gensdf/utils/sdf_fusion.py:194: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  samples[:, 0] = (overall_index // (N * N)) % N


Progress: 0.0%
Progress: 0.5%
Progress: 1.0%
Progress: 1.5%
Progress: 2.0%
Progress: 2.4%
Progress: 2.9%
Progress: 3.4%
Progress: 3.9%
Progress: 4.3%
Progress: 4.8%
Progress: 5.3%
Progress: 5.8%
Progress: 6.2%
Progress: 6.7%
Progress: 7.2%
Progress: 7.7%
Progress: 8.2%
Progress: 8.6%
Progress: 9.1%
Progress: 9.6%
Progress: 10.1%
Progress: 10.5%
Progress: 11.0%
Progress: 11.5%
Progress: 12.0%
Progress: 12.4%
Progress: 12.9%
Progress: 13.4%
Progress: 13.9%
Progress: 14.4%
Progress: 14.8%
Progress: 15.3%
Progress: 15.8%
Progress: 16.3%
Progress: 16.7%
Progress: 17.2%
Progress: 17.7%
Progress: 18.2%
Progress: 18.6%
Progress: 19.1%
Progress: 19.6%
Progress: 20.1%
Progress: 20.6%
Progress: 21.0%
Progress: 21.5%
Progress: 22.0%
Progress: 22.5%
Progress: 22.9%
Progress: 23.4%
Progress: 23.9%
Progress: 24.4%
Progress: 24.8%
Progress: 25.3%
Progress: 25.8%
Progress: 26.3%
Progress: 26.8%
Progress: 27.2%
Progress: 27.7%
Progress: 28.2%
Progress: 28.7%
Progress: 29.1%
Progress: 29.6%
Progress: 30.